# Autenticidad, 
# los Mensajes Secretos,
# y Seguridad



<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/04_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparar el Espacio (Prepare the Space)

In [ ]:
! pip install eciespy eth_keys gspread oauth2client
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! pip install --upgrade gspread

# Historias

<img src="https://github.com/ProfDoeg/Colegio_Invisible/raw/main/img/barb_2.jpeg" width="250"/>


*¡Verdezuela, Verdezuela,\
Suéltame tu cabellera!*\
-La Bruja

*El cifrado es una poderosa defensa arma para gente libre.*\
(*Encryption is a powerful defensive weapon for free people.*)\
-Esther Dyson




# Aplicar Criptografía Para Crear un Ensamblaje de Mensajería Seguro 
# (Apply Cryptography to Create a Secure Messaging Assembly)

## Esquema (Outline) 

Aplicar criptografía para crear un tablero de mensajes securos.
***
Apply cryptography to build a secure messaging board.

## Metas (Goals)

- crear y guardar claves\
(create and save keys)
- descifrar el token de acceso a la hoja de cálculo\
(decrypt spreadsheet access token)
- entender `gspread`\
(understand `gspread`)
- publicar: nombre, clave pública, mensaje público y firma\
(post: name, public key, public message, and signature)
- firmar mensaje\
(sign message)
- verificar mensaje\
(verify message)
- escribir mensaje a otros participantes\
(write message to other participants)
- escribir un mensaje encriptado a otro participante\
(write encrypted message to other participant)
- escribir mensaje firmado y encriptado\
(write signed and encrypted message)


# Hacer Nuevas Claves (Make New Keys)




In [ ]:
! mkdir llaves

In [ ]:
! ls

In [ ]:
! python Colegio_Invisible/scripts/ecc_generate.py llaves/prvkey1.eck ''

In [ ]:
! python Colegio_Invisible/scripts/ecc_pubkey_extract.py llaves/prvkey1.eck '' llaves/pubkey1.eck '' 

# Usar Guion Para Acceder Clave Pública (Use Script to Access Public Key)

In [ ]:
ret=!{ "python Colegio_Invisible/scripts/ecc_keydump.py llaves/pubkey1.eck ''"}
pubkey1=ret[-1]
pubkey1

# Gestionar el Acceso a Google Sheets (Manage Access to Google Sheets)


## Descifrar Credenciales de API (Decrypt API Credentials)

$\color{white}{\text{c0leg101nv1s1ble}}$

Usar la contraseña de arriba para descifrar `client_secret.aes`
***
Use the above password to decrypt `client_secret.aes`





In [ ]:
! python Colegio_Invisible/scripts/aes_decrypt.py c0leg101nv1s1ble Colegio_Invisible/secrets/client_secret.aes Colegio_Invisible/secrets/client_secret.json

In [ ]:
! ls Colegio_Invisible/secrets

## `gspread` 🪴



`gspread` es un módulo de Python para la interacción programática con Google Sheets.\
Todos los participantes tendrán acceso a la misma hoja de cálculo en la nube.
***
`gspread` is a python module for programmatic interaction with Google Sheets.\
All the participants will have access to the same spreadsheet in the cloud.



https://docs.gspread.org/en/latest/user-guide.html

Usar las credenciales descifradas para autenticar y obtener acceso API a la hoja de cálculo
***
Use the decrypted credentials to authenticate and gain API access to the spreadsheet

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Colegio_Invisible/secrets/client_secret.json', scope)
client = gspread.authorize(creds)
sheets = client.open("signatures")

# Sheets

Aquí podemos ver la estructura de la hoja de cálculo y todas las capas o `Worksheet`
***
Here we can see the structure of the spreadsheet and all of the layers or `Worksheet`

In [ ]:
sheets.worksheets()

In [ ]:
sheet=sheets.worksheets()[0]
sheet1=sheets.worksheets()[1]
sheet2=sheets.worksheets()[2]
sheet3=sheets.worksheets()[3]
sheet,sheet1,sheet2,sheet3

## Sheet 0 : Registro (Roster)

Aislar la primera hoja donde consultamos la primera fila
***
Isolate the first sheet where we query the first row

In [ ]:
sheet.row_values(1)

**PELIGRO: NO EJECUTAR**
***
**DANGER: DO NOT RUN**

In [ ]:
sheet.clear()

sheet.append_row(["name", "pubkey","statement","signature"])


### Tomar el Registro (Make the Roster)

Publicar (Publish):

- `name` nombre de participante\
(`name` title of participant)
- `pubkey` clave público de participante en hexadecimal\
(`pubkey` public key of participant in hex)
- `statement` la declaración firmada por el participante\
(`statement` data signed by participant)
- `signature` firma ecc de la declaración\
(`signature` ecc signature of the statement)

#### Entrada Datos (Input Data)

In [ ]:
my_name=input('input name')
my_statement=input('input statement')
open('llaves/statement1.txt','wb').write(my_statement.encode())

In [ ]:
! cat llaves/statement1.txt

Usar script para firmar la declaración
***
Use script to sign statement

In [ ]:
ret = !{ "python Colegio_Invisible/scripts/ecc_sign.py llaves/prvkey1.eck '' llaves/statement1.txt llaves/sig1.sig"}
sig1=ret[-1]
sig1

#### Publicar Datos (Publish Data)


In [ ]:
peeps=sheet.col_values(1)[1:]
datas=[my_name,pubkey1,my_statement,sig1]
if not (my_name in peeps) :
  sheet.append_row(datas)
else:
  row=peeps.index(my_name)+2
  for i,value in enumerate(datas[1:]):
    sheet.update_cell(row=row,col=i+2,value=value)

## Leer Sheet 0 (Read Sheet 0)

In [ ]:
import pandas as pd

dataframe = pd.DataFrame(sheet.get_all_records())
dataframe

## Verificar Sheet0 Firma/Declaración/Clave Pública (Verify Sheet0 Signature/Statement/PublicKey)

Usar la biblioteca de criptografía `ecies` directamente sin usar los scripts externos.
***
Use the `ecies` cryptography library directly without using the external scripts.

- importar la biblioteca\
 (import library)
- definir la función de verificación\
(define verification function)
- probar la función\
(test function)
- aplicar a cada fila de la tabla\
(apply to every row of table)

In [ ]:
import eth_keys
import ecies

def verify_message(sig,statement,pubkey):
  try:
    return eth_keys.datatypes.Signature(bytes.fromhex(sig)).verify_msg(statement.encode() if type(statement)!=bytes else statement,eth_keys.keys.PublicKey(bytes.fromhex(pubkey)))
  except Exception as E:
    return E
verify_message(sig1,my_statement,pubkey1)

In [ ]:
dataframe.set_index('name').apply(lambda row: verify_message(row.signature,row.statement,row.pubkey),axis=1)

# Configurar Tableros de Comunicación: Sheets 1-3
# (Setup Communication Boards: Sheets 1-3)


In [ ]:
#ONLY FOR DDOG MC
hojas=[sheet1,sheet2,sheet3]
participants = dataframe.name.tolist()
df=pd.DataFrame('None',columns=participants,index=participants)

def fill_sheets(h,dfrm):
  for hoja in h:
    hoja.clear()
    hoja.update([dfrm.columns.values.tolist()] + dfrm.values.tolist())

fill_sheets(hojas,df)

df

# Mensajes Abiertos: Sheet 1 (Open Messages: Sheet 1)

***
**SHEET 1**: Este es un lugar para charlas públicas informales.\
 Todos los mensajes son visibles y cualquier participante puede escribir en cualquier celda. No hay seguridad.
***
**SHEET 1**: This is a place for casual public chats.\
All messages are visible and any participant can write to any cell. There is no security.
***
- la fila representa el remitente\
(the row represents the sender)
- la columna representa  el destinatario\
(column represents the recipient)

In [ ]:
def sheet2df(she):
  datafr = pd.DataFrame(she.get_all_records())
  datafr.index=datafr.columns
  return datafr

In [ ]:
dataframe1 = sheet2df(sheet1)
dataframe1

## Crear Widget para Seleccionar el Destinatario del Mensaje (Create Widget for Selecting Message Recipient) 

In [ ]:
import ipywidgets as widgets
w=widgets.Dropdown(
    options=participants,
    value='DrDoeg',
    description='Recipient:',
    disabled=False,
)

In [ ]:
#variable returns dropdown menu
#change value by clicking on it
w

In [ ]:
def dest():
  print(f'Vos eres:')
  print(f'You are: {my_name}')
  print('~'*20)
  print('Selectar destinatario del mensaje')
  print('Select a message recipient')
  return w

dest()

In [ ]:
#
w.value

## Crear y Publicar Mensaje (Create and Post Message)

- Seleccionar contenedor\
(Select recipient)
- Escribir un mensaje\
(Write message)
- Publicar mensaje\
(Publish message)
- Ver tablero de mensajes\
(View message board)

In [ ]:
dest()

In [ ]:
p_dict=dict([ (e,i) for i,e in enumerate(participants)])
message=input('send a message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet1.update_cell(*write_cell, message);

In [ ]:

dataframe1 = sheet2df(sheet1)
dataframe1

## Leer Tus Mensajes (Read Your Messages)

In [ ]:
dataframe1[my_name]

# Mensajes Cifradas (Encrypted Messages) : Sheet2

In [ ]:
dataframe2 = sheet2df(sheet2)
dataframe2

In [ ]:
def encrypt_message(hexPubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    return ecies.encrypt(hexPubKey,message).hex()

In [ ]:
dest()

In [ ]:
message=input('send an encrypted message')
write_cell=(p_dict['michelle']+2,p_dict[w.value]+1)
sheet2.update_cell(*write_cell, encrypt_message(dataframe.set_index('name').pubkey[w.value],message));

In [ ]:
dataframe2 = sheet2df(sheet2)
dataframe2

In [ ]:
def import_privKey(path,password):
  import hashlib
  if type(password)!=bytes:
    password=password.encode()
  f=open(path,'rb')
  decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
  privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
  f.close()
  return privKey

def decrypt_message(privKey,cipherhex):
    try:
      ciphertext=bytes.fromhex(cipherhex)
      return ecies.decrypt(privKey.to_hex(),ciphertext)
    except:
      return None

In [ ]:
decrypt_message(import_privKey('llaves/prvkey1.eck',''),dataframe2[my_name][my_name])

In [ ]:
k=import_privKey('llaves/prvkey1.eck','')
tr=dataframe2[my_name].apply(lambda x: decrypt_message(k,x))
tr

# Firmada y Cifrada Mensajes (Signed and Encrypted Messages) : Sheet3

Escribir funciones (Write functions):
- sign
- sign and encrypt
- decrypt and verify

In [ ]:
def sign_message(privKey,message):
    if type(message)!=bytes:
        message=message.encode()
    signature= privKey.sign_msg(message)
    return signature.to_bytes().hex()

In [ ]:
def sign_encrypt_message(privKey,hexpubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    byte_enc=bytes.fromhex(encrypt_message(hexpubKey,message))
    signature= privKey.sign_msg(byte_enc)
    return signature.to_bytes().hex()+byte_enc.hex()

In [ ]:
def decrypt_verify(pubkey,privkey,block):
  try:
    ver=verify_message(block[:130],bytes.fromhex(block[130:]),pubkey)
    return decrypt_message(privkey,block[130:]),ver
  except:
    return None

In [ ]:
dest()

In [ ]:
message=input('send a signed and encrypted message to '+w.value)
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
signed_enc_message=sign_encrypt_message(k,pubkey1,message)
sheet3.update_cell(*write_cell,signed_enc_message);

In [ ]:
dataframe3 = sheet2df(sheet3)
dataframe3

## Check Signed and Encrypted Messages

In [ ]:
pubkey_name=dataframe.set_index('name')['pubkey']
dataframe3[[my_name]].reset_index().apply(lambda x: decrypt_verify(pubkey_name[x['index']],k,x[my_name])  ,axis=1)

In [ ]:
pubkey_name

In [ ]:
dataframe3[[my_name]].reset_index()

# Funciones

```python
sign_message(privKey,message)
verify_message(sig,statement,pubkey)
encrypt_message(hexPubKey,message)
decrypt_message(privKey,cipherhex)
sign_encrypt_message(privKey,hexpubKey,message)
decrypt_verify(pubkey,privkey,block)
```